In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [50]:
#Import data
df_2019 = pd.read_csv('data_2019.csv', low_memory = False)
df_2020 = pd.read_csv('data_2020.csv', low_memory = False)

df_full = pd.concat([df_2019, df_2020])

In [51]:
#Rename columns
rename_columns = {'datetTimeFirstDeliveryMoment': 'dateTimeFirstDeliveryMoment',
                  'generalMatchClassification': 'detailedMatchClassification',
                  'detailedMatchClassification': 'generalMatchClassification'}

df_full = df_full.rename(columns = rename_columns)

In [52]:
#Transform dates to date-type
df_full['orderDate'] = pd.to_datetime(df_full['orderDate'])
df_full['cancellationDate'] = pd.to_datetime(df_full['cancellationDate'])
df_full['promisedDeliveryDate'] = pd.to_datetime(df_full['promisedDeliveryDate'])
df_full['shipmentDate'] = pd.to_datetime(df_full['shipmentDate'])
df_full['dateTimeFirstDeliveryMoment'] = pd.to_datetime(df_full['dateTimeFirstDeliveryMoment'])
df_full['startDateCase'] = pd.to_datetime(df_full['startDateCase'])
df_full['returnDateTime'] = pd.to_datetime(df_full['returnDateTime'])
df_full['registrationDateSeller'] = pd.to_datetime(df_full['registrationDateSeller'])

In [55]:
df_full.shape

(4779466, 38)

(returnDateTime < (CONVERT(date,dateTimeFirstDeliveryMoment)) AND dateTimeFirstDeliveryMoment IS NOT NULL AND returnDateTime IS NOT NULL) OR (shipmentDate > returnDateTime)

(cancellationDate > shipmentDate AND (cancellationReasonCode = 'CUST_FE' OR cancellationReasonCode = 'CUST_CS'))

In [86]:
#(df['returnDateTime'] < df['dateTimeFirstDeliveryMoment'] & df['dateTimeFirstDeliveryMoment'].notnull() & df['returnDateTime'].notnull())
#(df['cancellationDate'] > df['shipmentDate'] & (df['cancellationReasonCode'] == 'CUST_FE' | df['cancellationReasonCode'] == 'CUST_CS'))
df = df_full.drop(df_full[(df_full['startDateCase'] < df_full['orderDate']) | 
                   (df_full['cancellationDate'] < df_full['orderDate']) |
                   (df_full['promisedDeliveryDate'] < df_full['orderDate']) |
                   (df_full['shipmentDate'] < df_full['orderDate']) |
                   (df_full['dateTimeFirstDeliveryMoment'].dt.normalize() < df_full['orderDate']) |
                   (df_full['returnDateTime'] < df_full['orderDate']) |
                   (df_full['orderDate'] < df_full['registrationDateSeller']) |
                   (df_full['cancellationDate'] > df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) |
                   (df_full['cancellationDate'] > df_full['returnDateTime']) |
                   (df_full['shipmentDate'] > df_full['returnDateTime']) |
                   (df_full['shipmentDate'] > df_full['dateTimeFirstDeliveryMoment'].dt.normalize()) |
                   (df_full['registrationDateSeller'].isnull()) |
                   (df_full['promisedDeliveryDate'].isnull())].index)

#Sort rows on orderDate and create new index
df = df.sort_values(by = 'orderDate')
df = df.reset_index(drop = True)

In [100]:
#df['Discounted_Price'] = df.apply(lambda row: row.Cost - (row.Cost * 0.1), axis = 1) 

#Create new variables related to periods (days difference)
df['caseDays'] = (df['startDateCase'] - df['orderDate']).dt.days
df['returnDays'] = (df['returnDateTime'] - df['orderDate']).dt.days
df['cancellationDays'] = (df['cancellationDate'] - df['orderDate']).dt.days
df['actualDeliveryDays'] = (df['dateTimeFirstDeliveryMoment'].dt.normalize() - df['orderDate']).dt.days
df['shipmentDays'] = (df['shipmentDate'] - df['orderDate']).dt.days
df['partnerSellingMonths'] = (df['orderDate'] - df['registrationDateSeller']).dt.days
df['promisedDeliveryDays'] = (df['promisedDeliveryDate'] - df['orderDate']).dt.days

#Time related variables
df['orderYear'] = df['orderDate'].dt.year
df['orderMonth'] = df['orderDate'].dt.month
df['orderWeekday'] = df['orderDate'].dt.weekday
df['orderCorona'] = df['orderDate'].apply(lambda x: True if x > '2020-03-20' else False)

#Create dummy variables for weeday and months
df['orderMonday'] = df['orderWeekday'].apply(lambda x: True if x == 1 else False)
df['orderTuesday'] = df['orderWeekday'].apply(lambda x: True if x == 2 else False)
df['orderWednesday'] = df['orderWeekday'].apply(lambda x: True if x == 3 else False)
df['orderThursday'] = df['orderWeekday'].apply(lambda x: True if x == 4 else False)
df['orderFriday'] = df['orderWeekday'].apply(lambda x: True if x == 5 else False)
df['orderSaturday'] = df['orderWeekday'].apply(lambda x: True if x == 6 else False)
df['orderSunday'] = df['orderWeekday'].apply(lambda x: True if x == 7 else False)

df['orderJanuary'] = df['orderMonth'].apply(lambda x: True if x == 1 else False)
df['orderFebruary'] = df['orderMonth'].apply(lambda x: True if x == 2 else False)
df['orderMarch'] = df['orderMonth'].apply(lambda x: True if x == 3 else False)
df['orderApril'] = df['orderMonth'].apply(lambda x: True if x == 4 else False)
df['orderMay'] = df['orderMonth'].apply(lambda x: True if x == 5 else False)
df['orderJune'] = df['orderMonth'].apply(lambda x: True if x == 6 else False)
df['orderJuly'] = df['orderMonth'].apply(lambda x: True if x == 7 else False)
df['orderAugust'] = df['orderMonth'].apply(lambda x: True if x == 8 else False)
df['orderSeptember'] = df['orderMonth'].apply(lambda x: True if x == 9 else False)
df['orderOctober'] = df['orderMonth'].apply(lambda x: True if x == 10 else False)
df['orderNovember'] = df['orderMonth'].apply(lambda x: True if x == 11 else False)
df['orderDecember'] = df['orderMonth'].apply(lambda x: True if x == 12 else False)

#Other variables
df['productTitleLength'] = len(df['productTitle'])

df['countryCodeNL'] = df['countryCode'].apply(lambda x: True if x == 'NL' else False)
df['fulfilmentByBol'] = df['fulfilmentType'].apply(lambda x: True if x == 'FBB' else False)
df['countryOriginNL'] = df['countryOriginSeller'].apply(lambda x: True if x == 'NL' else False)
df['countryOriginBE'] = df['countryOriginSeller'].apply(lambda x: True if x == 'BE' else False)
df['countryOriginDE'] = df['countryOriginSeller'].apply(lambda x: True if x == 'DE' else False)

#determinantClassification

<ipython-input-100-f0985cb99479>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['caseDays'] = (df['startDateCase'] - df['orderDate']).dt.days
<ipython-input-100-f0985cb99479>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['returnDays'] = (df['returnDateTime'] - df['orderDate']).dt.days
<ipython-input-100-f0985cb99479>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [105]:
#Change type of columns
dtype = {'calculationDefinitive': bool,
         'noCancellation': bool,
         'noCase': bool,
         'hasOneCase': bool,
         'hasMoreCases': bool,
         'noReturn': bool}

df = df.astype(dtype)
df.dtypes

In [108]:
def addPeriodColumns(df,X):
    """
    Function to create dynamic columns based on the prediction period.
    X = number of days after order date
    """
    
    dynamic_cols = ['caseDays','returnDays','cancellationDays','actualDeliveryDays']
    
    for col in dynamic_cols:
        
        dynamic_colname = col+'_'+str(X)
        df[dynamic_colname] = df[col].dt.days
        df[dynamic_colname] = df[dynamic_colname].apply(lambda x: x if x <= X else None)
        
    return df

In [204]:
def addProductCount_HARD(df):
    """
    Fuction to add a column to your dataframe which counts the number of occurances of
    the productId uptil that date.
    """

    df = df.sort_values(by = ['productId','orderDate'])
    df = df.reset_index(drop = True)
    
    df_ = df[['productId','orderDate']]
    
    productCount = np.ones(df_.shape[0])
    
    previousID = None
    previousDate = None
    sameDate = False
    countSameDate = 1
    firstIndexSameDate = None

    for row in df_.itertuples():
        
        if (previousID == row[1]) and (previousDate != row[2]) and (sameDate == False):
            productCount[row[0]] = productCount[row[0]-1] + 1
            
            previousID = row[1]
            previousDate = row[2]
            
        elif (previousID == row[1]) and (previousDate != row[2]) and (sameDate == True):
            productCount[firstIndexSameDate:row[0]] = productCount[firstIndexSameDate-1] + countSameDate
            productCount[row[0]] = productCount[row[0]-1] + 1
            
            previousID = row[1]
            previousDate = row[2]
            
            sameDate = False
            firstIndexSameDate = None
            countSameDate = 1
            
        elif (previousID == row[1]) and (previousDate == row[2]) and (sameDate == False):
            sameDate = True
            firstIndexSameDate = row[0] - 1
            countSameDate += 1
            
            previousID = row[1]
            previousDate = row[2]
            
        elif (previousID == row[1]) and (previousDate == row[2]) and (sameDate == True):
            countSameDate +=1
            
            previousID = row[1]
            previousDate = row[2]
            
        elif (previousID != row[1]) and (sameDate == True):
            productCount[firstIndexSameDate:row[0]] = productCount[firstIndexSameDate-1] + countSameDate
            
            sameDate = False
            firstIndexSameDate = None
            countSameDate = 1
            
            previousID = row[1]
            previousDate = row[2]
            
        elif (previousID != row[1]) and (sameDate == False):
            previousID = row[1]
            previousDate = row[2]
    
    
    df['productCount'] = productCount
    
    return(df)

In [279]:
# More elegant but takes a very long time to solve

# def addProductCount_HARD(df):
#     """
#     Fuction to add a column to your dataframe which counts the number of occurances of
#     the productId uptil that date.
#     """

#     df_ = df[['productId','orderDate']].sort_values(by = ['productId','orderDate'])
#     df_ = df_.reset_index(drop = True)
    
#     productCount = np.ones(df_.shape[0])

#     for row in df_.itertuples():
        
#         if row[0] == 0:
#             continue
#         else:
#             productCount[row[0]] = df_[(df_['orderDate'] <= row[2]) & (df['productId'] == row[1])].shape[0]
               
#     df['productCount'] = productCount
    
#     return(df)

In [295]:
def addProductCount(df):
    """
    Fuction to add a column to your dataframe which counts the number of occurances of
    the productId uptil that date.
    """
    
    df_ = df[['productId','orderDate']].sort_values(by = ['productId','orderDate'])
    df_ = df_.reset_index(drop = True)
    
    productCount = np.ones(df_.shape[0])
    
    previousID = None
    previousDate = None
    
    for row in df_.itertuples():
        
        if row[0] == 0:
            previousID = row[1]
            
        elif (previousID == row[1]):
            countSameDate
            productCount[row[0]] = productCount[row[0] - 1] + 1
            previousID = row[1]
            
        else:
            previousID = row[1]
        
    df['productCount'] = productCount
    
    return(df)